# 1. Installing Dependencies

Run this file to install all dependencies needed by the Notebook. You can skip running this cell if you have runned this cell in the current environment previously.

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt
!jupyter nbextension enable --py widgetsnbextension
!jupyter serverextension enable voila --sys-prefix

# 2. Setting Environment Variables
<ul>
    <li> Make sure you have set your environment vairables in `var.env` file.
    <li> Pick the date range for your analysis
    <li> After resetting any environment variables, you need to restart the kernel because otherwise it will not be loaded by Jupyter. To restart, go to the menu 'Kernel' and choose 'Restart'
    <li> Run all the cells in this section
    <li> Make sure the environment variables are set correctly
<ul>

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv('var.env')

import os
from itables import init_notebook_mode
if os.getenv('IS_INTERACTIVE_TABLES_MODE') == 'TRUE':
    init_notebook_mode(all_interactive=True)

from IPython.display import display
import ipywidgets as widgets
from ipywidgets import HBox
start_date_picker = widgets.DatePicker(description='Start Date')
end_date_picker = widgets.DatePicker(description='End Date')
date_pickers = HBox(children=[start_date_picker, end_date_picker])
display(date_pickers)

In [ ]:
os.environ['START_DATE'] = str(start_date_picker.value)
os.environ['END_DATE'] = str(end_date_picker.value)
print('--- LOADED ENVIRONMENT VARIABLES ---')
print(f"INPUT_PROJECT_ID: {os.getenv('INPUT_PROJECT_ID')}")
print(f"INPUT_DATASET_ID: {os.getenv('INPUT_DATASET_ID')}")
print(f"INPUT_AUDIT_LOGS_TABLE_ID: {os.getenv('INPUT_AUDIT_LOGS_TABLE_ID')}")
print(f"IS_AUDIT_LOGS_TABLE_PARTITIONED: {os.getenv('IS_AUDIT_LOGS_INPUT_TABLE_PARTITIONED')}")
print(f"OUTPUT_PROJECT_ID: {os.getenv('OUTPUT_PROJECT_ID')}")
print(f"OUTPUT_DATASET_ID: {os.getenv('OUTPUT_DATASET_ID')}")
print(f"OUTPUT_TABLE_SUFFIX: {os.getenv('OUTPUT_TABLE_SUFFIX')}")
print(f"LOCATION: {os.getenv('LOCATION')}")
print(f"START_DATE: {os.getenv('START_DATE')}")
print(f"END_DATE: {os.getenv('END_DATE')}")

# 3. Creating Tables for Current Analysis

Run the cell below to create tables that is necessary for the analysis

In [4]:
from src.bq_query import BQQuery
try:
    BQQuery.create_functions_for_pipeline_analysis()
    BQQuery.create_tables_for_pipeline_analysis()
except Exception as e:
    print('Unable to create tables, do not continue with the analysis')
    print(e)

# 4. Getting Analysis Result

### Get the tables with highest discrepancy on write vs read frequency throughout the data warehouse
This will list down tables with the highest discrepancy on write vs read frequency.

1. Run the cell
2. Set the limit on how many tables you want to be displayed using the text box, please insert positive values only. 
3. Click 'Run' and wait until the result is retrieved.

In [5]:
import src.pipeline_analysis as pipeline_analysis
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

limited_imbalance_tables = []
limited_imbalance_tables_df = pd.DataFrame()
def get_limited_imbalance_tables_df(limit):
    global limited_imbalance_tables, limited_imbalance_tables_df
    limited_imbalance_tables_df = pipeline_analysis.get_tables_read_write_frequency_df(limit)
    limited_imbalance_tables = limited_imbalance_tables_df['Table'].tolist()
    return limited_imbalance_tables_df

widgets.interact_manual.opts['manual_name'] = 'Run'
widgets.interact_manual(get_limited_imbalance_tables_df, limit= widgets.IntText(value=3))
;

interactive(children=(IntText(value=3, description='limit'), Button(description='Run', style=ButtonStyle()), O…

''

### Get the pipeline graph data of the table
This will generate a pipeline graph file, in HTML format, under `graph` directory. It may take sometime for this to run and generate.

1. Choose the table of interest, the table that you are interested to explore further by displaying its pipeline graph.
2. Click 'Run' and wait until the run is finished (indicated by non grayed-out box).
3. Run the next cell to display the graph

In [6]:
def visualise_table_pipelines(table):
    pipeline_analysis.display_pipelines_of_table(table)

widgets.interact_manual(visualise_table_pipelines, table = widgets.Dropdown(options=limited_imbalance_tables+ [''], value='', description='Table:'))
;

interactive(children=(Dropdown(description='Table:', index=3, options=('data-analytics-pocs.atos.temp_updates'…

''

### Display the pipeline graph of the table
Display the pipeline graph of the table. The thickness of the edges indicates the frequency compared to the rest of the edges in the current graph.

1. Run the cell to display the pipeline graph of the table in the iFrame below
2. You can click on the different nodes of the graph, each representing different tbales that are part of the pipeline of this table of interest. When you click on a node, it will display more information for this table.

In [11]:
from IPython.display import IFrame,HTML, display
display(IFrame('./pipeline_graph/index.html', width="1000", height="800"))